In [12]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd 

In [13]:
df=pd.read_excel('C:/Users/tian/Desktop/数据data/京东口红评论数据.xlsx',header=0)
df

,content,content_type
0,产品非常好，拿到的时候非常敬业阿，这个外观非常好看阿，东西非趁还是很好的，一如既往的很好这次...,pos
1,还是可以的老婆觉得贵，用了感觉也不是特别好，拿到的时候非常敬业阿，这个外观非常好看阿，东西非...,pos
2,这只的质地简直是我最喜欢的一支口红，没有之一，试完641立马下单了999，肉眼可见的润，而且...,pos
3,第一次买迪奥080也是第一次京东自营买！看某书分享的真伪辨别方法感觉是真的吧！老公送我的但还...,pos
4,产品质感：哑光，但是不干，很顺滑，超级舒服！！！质感超棒！！\n产品颜色：这只太好看，太温柔...,pos
...,...,...
1995,20号买的时候320块，才过一天都变成275块了，我****,neg
1996,"产品质感：这是个什么情况，出现这样一个现象"",""vcontent"":""{$%&产品质感：&%...",neg
1997,"其他特色：这就不是什么礼盒，只是个盒子。"",""vcontent"":""{$%&其他特色：&%$...",neg
1998,说句实话将近400的东西盒子里的东西也太简陋了吧，就一点絮絮子加支口红，三瓶小样，但是口红质...,neg


In [14]:
import jieba

In [15]:
data_good=df[df['content_type']=='pos'].content.to_list()

data_bad=df[df['content_type']!='pos'].content.to_list()

In [16]:
datas = ['data_good','data_bad']
samples=[]
for data in datas:
    for x in eval(data):
        str_=jieba.lcut(x)
        str_=filter(lambda x: len(x)>1,str_)
        samples.append((data,' '.join(str_)))


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\tian\AppData\Local\Temp\jieba.cache
Loading model cost 0.643 seconds.
Prefix dict has been built successfully.


In [17]:
import random
random.shuffle(samples)

In [18]:
y,X=zip(*samples)

In [19]:
from sklearn.model_selection import train_test_split

xtrain,xtest,ytrain,ytest = train_test_split(X,y)

In [20]:
# 停词表
stopwords=pd.read_csv(r'C:/Users/tian/Desktop/数据data/stopwords.txt',index_col=False,quoting=3,sep='\t',names=['stopword'],encoding='utf-8')
stopwords=stopwords['stopword'].values.tolist()

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


In [22]:
vec=CountVectorizer(stop_words=stopwords,max_features=60000).fit(xtrain) #词频统计  训练模型
tf_idf=TfidfVectorizer(stop_words=stopwords,max_features=60000).fit(xtrain) # TF-IDF 值越大越重要
# 词袋总特征数60000个，已经去掉停用词
xtrain_=vec.transform(xtrain)
xtest_=vec.transform(xtest)

xtrain_df=tf_idf.transform(xtrain)
xtest_df=tf_idf.transform(xtest)

In [23]:
from sklearn.naive_bayes import GaussianNB # 高斯朴素贝叶斯
from sklearn.naive_bayes import BernoulliNB # 伯努利朴素贝叶斯
from sklearn.naive_bayes import MultinomialNB #多项式朴素贝叶斯
from sklearn.naive_bayes import ComplementNB #补集朴素贝叶斯

In [24]:
nb=MultinomialNB().fit(xtrain_,ytrain)
nb_df=MultinomialNB().fit(xtrain_df,ytrain)

In [25]:
print(nb.score(xtest_,ytest))
print(nb_df.score(xtest_df,ytest))

0.9
0.924


In [26]:
text='客服太懒了'
texts=[' '.join(jieba.lcut(text))]
tex=vec.transform(texts)

In [27]:
nb_df.predict(tex)

array(['data_bad'], dtype='<U9')

In [28]:
nb_df.predict_proba(tex)

array([[0.93563425, 0.06436575]])

In [29]:
xt=vec.transform(X)
nb.predict(xt)[:10]

array(['data_good', 'data_bad', 'data_good', 'data_good', 'data_bad',
       'data_good', 'data_good', 'data_bad', 'data_good', 'data_good'],
      dtype='<U9')

In [30]:
nb.predict(tex)[0]

'data_bad'

In [31]:
nb.predict_proba(tex)

array([[0.958947, 0.041053]])